In [1]:
import rdflib
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF
import spotlight
import urllib, json
import pandas as pd
from collections import Counter

In [47]:
disease = URIRef('http://dbpedia.org/ontology/Disease')
drug = URIRef('http://dbpedia.org/ontology/Drug')
symp = URIRef('http://dbpedia.org/resource/Category:Symptoms_and_signs')
anat = URIRef('http://dbpedia.org/ontology/AnatomicalStructure')
medicin = URIRef('http://dbpedia.org/page/Medical_diagnosis')
therapy = URIRef('http://dbpedia.org/page/Therapy')
group = URIRef('http://dbpedia.org/page/Category:Human_development')

In [148]:
def spalq_condition_ds(con):
    g = rdflib.Graph()
    result = g.parse('cknns/'+con+'.ttl', format='turtle')
    #print (result)
    querystring = "SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {"
    querystring = querystring+"?d <"+RDF.type+"> <"+disease+"> . " 
    querystring = querystring+"?s <"+RDF.type+"> <"+symp+"> . }" 
    #print(querystring)
    qres = g.query(querystring)
    
    #print (qres)
    d = []
    s = []
    
    for row in qres:
        di = str(row.asdict()['d'].toPython()).split('/')[-1]
        if di not in d:
            d.append(di)
        si = str(row.asdict()['s'].toPython()).split('/')[-1]
        if si not in s:
            s.append(si)
    g.close()
    return d,s

In [149]:
def spalq_condition_pos(con):
    g = rdflib.Graph()
    result = g.parse('cknns/'+con+'.ttl', format='turtle')
    #print (result)
    querystring = "SELECT DISTINCT ?r WHERE {"
    querystring = querystring+"?r <"+RDF.type+"> <"+anat+"> . }" 
    #print(querystring)
    qres = g.query(querystring)
    
    #print (qres)
    r = []
    if not qres:
        return r
    else:
        for row in qres:
            ri = str(row.asdict()['r'].toPython()).split('/')[-1]
            if ri not in r:
                r.append(ri)
    g.close()
    return r

In [150]:
def spalq_condition_drug(con):
    g = rdflib.Graph()
    result = g.parse('cknns/'+con+'.ttl', format='turtle')
    #print (result)
    querystring = "SELECT DISTINCT ?r WHERE {"
    querystring = querystring+"?r <"+RDF.type+"> <"+drug+"> . }" 
    #print(querystring)
    qres = g.query(querystring)
    
    #print (qres)
    r = []
    if not qres:
        return r
    else:
        for row in qres:
            ri = str(row.asdict()['r'].toPython()).split('/')[-1]
            if ri not in r:
                r.append(ri)
    g.close()
    return r

In [151]:
def spalq_condition_treat(con):
    g = rdflib.Graph()
    result = g.parse('cknns/'+con+'.ttl', format='turtle')
    #print (result)
    querystring = "SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {"
    querystring = querystring+"?r <"+RDF.type+"> <"+therapy+"> . }" 
    #print(querystring)
    qres = g.query(querystring)
    
    #print (qres)
    r = []
    if not qres:
        return r
    else:
        for row in qres:
            ri = str(row.asdict()['r'].toPython()).split('/')[-1]
            if ri not in r:
                r.append(ri)
    g.close()
    return r

In [152]:
def spalq_condition_medic(con):
    g = rdflib.Graph()
    result = g.parse('cknns/'+con+'.ttl', format='turtle')
    #print (result)
    querystring = "SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {"
    querystring = querystring+"?r <"+RDF.type+"> <"+medicin+"> . }" 
    #print(querystring)
    qres = g.query(querystring)
    
    #print (qres)
    r = []
    if not qres:
        return r
    else:
        for row in qres:
            ri = str(row.asdict()['r'].toPython()).split('/')[-1]
            if ri not in r:
                r.append(ri)
    g.close()
    return r

In [159]:
def spalq_condition_people(con):
    g = rdflib.Graph()
    result = g.parse('cknns/'+con+'.ttl', format='turtle')
    #print (result)
    querystring = "SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {"
    querystring = querystring+"?r <"+RDF.type+"> <"+group+"> . }" 
    #print(querystring)
    qres = g.query(querystring)
    
    #print (qres)
    r = []
    if not qres:
        return r
    else:
        for row in qres:
            ri = str(row.asdict()['r'].toPython()).split('/')[-1]
            if ri not in r:
                r.append(ri)
    g.close()
    return r

In [336]:
Diseaselist = ['asthma','breast-cancer','chickenpox','flu','gallstone','heart-attack','hypertension','lung-cancer','pneumonia','urinary-incontinence','urinary-tract-infection','stroke','glaucoma']

In [337]:
#prediction v1.0

In [333]:
def predict_ranking(symptom,pre_disease,agegroup,sex):
    p_df = pd.DataFrame(columns=['Condition','Disease', 'Symptom', 'Drug', 'Position','Medic','Treat', 'Group'])

    i =0
    for dis in Diseaselist:
        d,s = spalq_condition_ds(dis)
    #print (d,s)
        t = spalq_condition_treat(dis)
    #print ('treat: ', t)
        p = spalq_condition_pos(dis)
    #print('position: ', p)
        m = spalq_condition_medic(dis)
    #print('medic: ', m)
        r = spalq_condition_drug(dis)
    #print('drug: ', r)
        gr = spalq_condition_people(dis)
    #print('people: ', gr)
    #d,r,s,p,m,t,a = spalq_condition(dis)
        p_df.loc[i] = [dis,d,s,r,p,m,t,gr]
        i=i+1
    scoredict ={}
    matchdict = {}
    symtcount ={}
    i = 0 
    #print (agegroup)
    #symptom ranking
    for sym in p_df['Symptom']:
        avsym = 0
        for s in sym:
            for symi in symptom:
                if symi == s:
                    avsym = avsym+1
            symtcount[s]=avsym
            #print (s, avsym)
        for s in sym:
            if s in symptom:
                c = symtcount[s]
                #if(p_df['Condition'][i]=='urinary-incontinence'):
                    #print(s, p_df['Condition'][i])
                if p_df['Condition'][i] in scoredict:
                    x = scoredict.get(p_df['Condition'][i])
                    scoredict[p_df['Condition'][i]]=x+1/c
                    y = matchdict.get(p_df['Condition'][i])
                    matchdict[p_df['Condition'][i]]=y+1
                else:
                    scoredict[p_df['Condition'][i]]=1/c
                    matchdict[p_df['Condition'][i]]=1
        i=i+1
    j = 0
    discount = {}
    for dis in p_df['Disease']:
        avsym = 0
        for d in dis:
            for disi in pre_disease:
                if disi == d:
                    avsym = avsym+1
            discount[d]=avsym
            #print (d, avsym)
        for d in dis:
            if d in pre_disease:
                c = discount[d]
                #if(p_df['Condition'][j]=='urinary-incontinence'):
                    #print(d, p_df['Condition'][j])
                if p_df['Condition'][j] in scoredict:
                    x = scoredict.get(p_df['Condition'][j])
                    scoredict[p_df['Condition'][j]]=x+1/c
                    y = matchdict.get(p_df['Condition'][j])
                    matchdict[p_df['Condition'][j]]=y+1
                else:
                    scoredict[p_df['Condition'][j]]=1/c
                    matchdict[p_df['Condition'][j]]=1
        j=j+1
    k=0
    agecount = {}
    for age in p_df['Group']:
        avsym = 0
        for a in age:
            for agi in agegroup:
                if agi == a:
                    avsym = avsym+1
            for sexi in sex:
                if sexi == a:
                    avsym = avsym+1
            agecount[a] = avsym
        for a in age:
            #print (a,agegroup)
            if a in agegroup:
                c = agecount[a]
                #if(p_df['Condition'][k]=='urinary-incontinence'):
                    #print(a, p_df['Condition'][k])
                if p_df['Condition'][k] in scoredict:
                    x = scoredict.get(p_df['Condition'][k])
                    scoredict[p_df['Condition'][k]]=x+1/c
                    y = matchdict.get(p_df['Condition'][k])
                    matchdict[p_df['Condition'][k]]=y+1
                else:
                    scoredict[p_df['Condition'][k]]=1/c
                    matchdict[p_df['Condition'][k]]=1
            if a in sex:
                c = agecount[a]
                #if(p_df['Condition'][k]=='urinary-incontinence'):
                    #print(a, p_df['Condition'][k])
                if p_df['Condition'][k] in scoredict:
                    x = scoredict.get(p_df['Condition'][k])
                    scoredict[p_df['Condition'][k]]=x+1/c
                    y = matchdict.get(p_df['Condition'][k])
                    matchdict[p_df['Condition'][k]]=y+1
                else:
                    scoredict[p_df['Condition'][k]]=1/c
                    matchdict[p_df['Condition'][k]]=1
        k=k+1
    normlisedict={}
    #print (scoredict)
    for x, y in scoredict.items():
            normlisedict[x]=y/4
    ordereddict= {k: v for k, v in sorted(normlisedict.items(), key=lambda item: item[1], reverse=True)}
    return ordereddict
    

In [334]:
#testing example:

In [338]:
ranking = predict_ranking(['Wheeze','Overactive_bladder'],['Schizophrenia','Blood_pressure'],['Ageing'],['Male'])

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> .  does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http:/

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> .  does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/0

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/resource/Category:Symptoms_and_signs> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Therapy does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Therapy does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Therapy> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < do

...

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Medical_diagnosis does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Medical_diagnosis> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Drug does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org

SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Drug does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Drug does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?r WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Drug> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Category:Human_development does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Category:Human_development does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?r <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/page/Category:Human_development> . } does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT D

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> .  does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http:/

SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> .  does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s < does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Disease> . ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type does not look like a valid URI, trying to serialize this will break.
SELECT DISTINCT ?d ?r ?s ?p ?m ?t ?k WHERE {?d <http://www.w3.org/

In [339]:
ranking

{'urinary-incontinence': 0.875,
 'asthma': 0.75,
 'pneumonia': 0.75,
 'glaucoma': 0.625,
 'breast-cancer': 0.5,
 'chickenpox': 0.5,
 'lung-cancer': 0.5,
 'stroke': 0.375,
 'heart-attack': 0.25,
 'hypertension': 0.25}